<a href="https://colab.research.google.com/github/mnijhuis-dnb/Artificial_Intelligence_and_Machine_Learning_for_SupTech/blob/main/Tutorial%208%20Explainable%20AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Artificial Intelligence and Machine Learning for SupTech  
Tutorial 8: Explainable AI 

*	Permutation importance with ELI5
*	Partial dependency plots
*	Shapley values

<br/>

15 March 2023  

**Instructors**  
Prof. Iman van Lelyveld (iman.van.lelyveld@vu.nl)<br/>
Dr. Michiel Nijhuis (m.nijhuis@dnb.nl)  

### Data loading

For this tutorial we will use the US census income dataset: 
https://archive.ics.uci.edu/ml/datasets/Census-Income+%28KDD%29

We will start with the installation of the packages we are going to use

In [ ]:
!pip install shap
!pip install eli5
!pip install --no-dependencies pdpbox

In [ ]:
import pandas as pd
import numpy as np
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Downloading and installing the data which will be used

In [ ]:
!gdown 131MoEFF9OEY9x61yMi_2fZmzJnt4P4qu

Reading in the data

In [ ]:
dtypes = [
	('age', 'float32'), ('class of worker', 'category'), ('detailed industry recode', 'category'), 
	('detailed occupation recode', 'category'), ('education', 'category'), ('wage per hour', 'float32'), 
	('enroll in edu inst last wk', 'category'), ('marital stat', 'category'), ('major industry code', 'category'), 
	('major occupation code', 'category'), ('race', 'category'), ('hispanic origin', 'category'), 
	('sex', 'category'), ('member of a labor union', 'category'), ('reason for unemployment', 'category'), 
	('full or part time employment stat', 'category'), ('capital gains', 'float32'), 
	('capital losses', 'float32'), ('dividends from stocks', 'float32'), ('tax filer stat', 'category'), 
	('region of previous residence', 'category'), ('state of previous residence', 'category'), 
	('detailed household and family stat', 'category'), ('detailed household summary in household', 'category'), 
	('instance weight_ignore', 'float32'), ('migration code-change in msa', 'category'), 
	('migration code-change in reg', 'category'), ('migration code-move within reg', 'category'), 
	('live in this house 1 year ago', 'category'), ('migration prev res in sunbelt', 'category'), 
	('num persons worked for employer', 'float32'),	('family members under 18', 'category'), 
	('country of birth father', 'category'), ('country of birth mother', 'category'), 
	('country of birth self', 'category'), ('citizenship', 'category'), ('own business or self employed', 'category'), 
	('fill inc questionnaire for veteran\'s admin', 'category'), ('veterans benefits', 'category'), 
	('weeks worked in year', 'float32'), ('year', 'category'), ('targets', 'category')]

raw_data = pd.read_csv('/content/census-income.data', names=[d[0] for d in dtypes], dtype=dict(dtypes))

edu_code = {"Children": 0,
            "Less than 1st grade": 1,  
            "1st 2nd 3rd or 4th grade": 2,  
            "5th or 6th grade": 3, 
            "7th and 8th grade": 4, 
            "9th grade": 5, 
            "10th grade": 6, 
            "11th grade": 7, 
            "12th grade no diploma": 8, 
            "High school graduate": 9,
            "Some college but no degree": 10,
            "Associates degree-academic program": 11, 
            "Associates degree-occup /vocational": 12,
            "Bachelors degree(BA AB BS)": 13,
            "Masters degree(MA MS MEng MEd MSW MBA)": 14,  
            "Prof school degree (MD DDS DVM LLB JD)": 15,
            "Doctorate degree(PhD EdD)": 15}
raw_data['education-num'] = np.array([edu_code[v.strip()] for v in raw_data['education']]).astype('float32')
dtypes.append(('education-num','float32'))

targets, target_value = pd.factorize(raw_data['targets'])
raw_data = raw_data.drop(columns=['instance weight_ignore', 'targets'])

Some data cleanup

In [ ]:
# remove some of the data to make it calculate quicker
data = raw_data.drop(columns=['detailed industry recode', 'detailed occupation recode', 
                              'major industry code', 'major occupation code', 'education', 
                              'country of birth father',  'country of birth mother', 
                              'country of birth self', 'state of previous residence', 
                              'detailed household and family stat'])
data_for_plot = data.copy()
binary_columns = ['sex','member of a labor union','live in this house 1 year ago',
                   'migration prev res in sunbelt','own business or self employed',
                   'fill inc questionnaire for veteran\'s admin','veterans benefits',
                   'year']

binary_data = data[binary_columns].copy()
categorical_data = data.select_dtypes(include=['category']).drop(columns=binary_data)
numerical_data = data.select_dtypes(include=['float32'])

binary_data[(binary_data==' 2') | (binary_data==' ?') | (binary_data==' Not in universe') | (binary_data==' Not in universe under 1 year old')] = np.nan
binary_data = (binary_data.apply(lambda x: pd.factorize(x)[0]))

categorical_data = categorical_data.apply(lambda x : pd.factorize(x)[0])
data = pd.DataFrame(np.hstack((categorical_data,numerical_data.values, binary_data.values)),
                     columns=list(categorical_data.columns) + list(numerical_data.columns) + list(binary_data.columns))
data = data.reindex(columns=data_for_plot.columns)

Making a test-train split

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(data, pd.Series(targets), random_state=99)

index = np.random.permutation(np.hstack((train_y.index.values[train_y==1],np.random.permutation(train_y.index.values[train_y==0])))[:2*np.sum(train_y==1)])
train_x_new = train_x.loc[index]
train_y_new = train_y.loc[index]

data.head()

Building a model...

In [ ]:
my_model = RandomForestClassifier(random_state=99, min_samples_split=500, max_leaf_nodes=50, oob_score=True).fit(train_x_new, train_y_new)
print(f'Model accuracy: {np.round(np.sum(my_model.predict(val_x)==val_y)/len(val_y)*100,1)}%')
print(f'Model recall: {np.round(np.sum((my_model.predict(val_x)==1) & (val_y==1))/np.sum(val_y==1)*100,1)}%')

### ELI5
Calculating feature importance with eli5:

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(my_model, random_state=1, scoring='recall').fit(train_x_new, train_y_new)
eli5.show_weights(perm, feature_names = data.columns.to_list(), top=50)

### Partial dependence plots
Now we will create a couple of partial dependence plots

In [ ]:
from matplotlib import pyplot as plt
from pdpbox import pdp, get_dataset, info_plots

pdp_education = pdp.pdp_isolate(model=my_model, dataset=val_x, model_features=data.columns.to_list(), cust_grid_points=np.r_[0:16], feature='education-num')

# plot it
pdp.pdp_plot(pdp_education, 'education-num')
plt.show()

In [ ]:
feature_to_plot = 'tax filer stat'
pdp_hours = pdp.pdp_isolate(model=my_model, dataset=val_x, model_features=data.columns.to_list(), cust_grid_points=np.r_[0:6], feature=feature_to_plot)

pdp.pdp_plot(pdp_hours, feature_to_plot)
plt.show()
pd.factorize(raw_data['tax filer stat'])[1].values

Also works with contour plots

In [ ]:
features_to_plot = ['age', 'education-num']
inter1  =  pdp.pdp_interact(model=my_model, dataset=val_x, model_features=data.columns.to_list(), features=features_to_plot)
pdp.pdp_interact_plot(pdp_interact_out=inter1, feature_names=features_to_plot, plot_type='contour')
plt.show()

### SHAP
Now we will take a look at the Shapley values for a single row

In [ ]:
row_number = 54
data_for_prediction = val_x.iloc[row_number]  
my_model.predict_proba(data_for_prediction.values.reshape(1, -1))

In [ ]:
explainer = shap.TreeExplainer(my_model)
shap_values = explainer.shap_values(data_for_prediction)
shap_values

SHAP also has nice visualizations for these values

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_plot.loc[val_x.index[row_number]])

For non tree based models we can use the KernelExplainer to get an approximate result

In [ ]:
shap.initjs()
k_explainer = shap.KernelExplainer(my_model.predict, shap.kmeans(val_x, 20))
k_shap_values = k_explainer.shap_values(data_for_prediction)
shap.force_plot(k_explainer.expected_value, k_shap_values, data_for_plot.loc[val_x.index[row_number]])

Calculating the Shapley values of 10000 random rows

In [ ]:
explainer = shap.TreeExplainer(my_model)
random_subset = val_x.sample(n=10000)
shap_values = explainer.shap_values(random_subset)

Now we can plot the results for multiple the rows

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1][:1000], data_for_plot.loc[random_subset.index[:1000]])

We can get a summary of the Shapley values as follows


In [ ]:
shap_values = explainer.shap_values(random_subset)
shap.summary_plot(shap_values[1], random_subset)

Or for the combined features

In [ ]:
shap.summary_plot(shap_values[0], random_subset, plot_type='bar')

Or for the dependence between 2 features

In [ ]:
shap.dependence_plot('education-num', shap_values[1], random_subset, interaction_index='weeks worked in year', x_jitter=1, dot_size=20)